In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Top_Bottom_N_Rows").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 13:32:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/15 13:32:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
data_student = [
    ("Michael", "Physics", 80, "P", 90),
    ("Michael", "Chemistry", 67, "P", 90),
    ("Michael", "Mathematics", 78, "P", 90),
    ("Nancy", "Physics", 30, "F", 80),
    ("Nancy", "Chemistry", 59, "P", 80),
    ("Nancy", "Mathematics", 75, "P", 80),
    ("David", "Physics", 90, "P", 70),
    ("David", "Chemistry", 87, "P", 70),
    ("David", "Mathematics", 97, "P", 70),
    ("John", "Physics", 33, "F", 60),
    ("John", "Chemistry", 28, "F", 60),
    ("John", "Mathematics", 52, "P", 60),
    ("Blessy", "Physics", 89, "P", 75),
    ("Blessy", "Chemistry", 76, "P", 75),
    ("Blessy", "Mathematics", 63, "P", 75),
]

schema = ["Name", "Subject", "Mark", "Status", "Attendance"]
df = spark.createDataFrame(data=data_student, schema=schema)
df.show()

+-------+-----------+----+------+----------+
|   Name|    Subject|Mark|Status|Attendance|
+-------+-----------+----+------+----------+
|Michael|    Physics|  80|     P|        90|
|Michael|  Chemistry|  67|     P|        90|
|Michael|Mathematics|  78|     P|        90|
|  Nancy|    Physics|  30|     F|        80|
|  Nancy|  Chemistry|  59|     P|        80|
|  Nancy|Mathematics|  75|     P|        80|
|  David|    Physics|  90|     P|        70|
|  David|  Chemistry|  87|     P|        70|
|  David|Mathematics|  97|     P|        70|
|   John|    Physics|  33|     F|        60|
|   John|  Chemistry|  28|     F|        60|
|   John|Mathematics|  52|     P|        60|
| Blessy|    Physics|  89|     P|        75|
| Blessy|  Chemistry|  76|     P|        75|
| Blessy|Mathematics|  63|     P|        75|
+-------+-----------+----+------+----------+



- Create Rank() with each group of Name

In [3]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [7]:
windowDept = Window.partitionBy("Name").orderBy(col("Mark").desc())

In [8]:
df2 = df.withColumn("row", row_number().over(windowDept)).orderBy("Name", "row")
df2.show()

+-------+-----------+----+------+----------+---+
|   Name|    Subject|Mark|Status|Attendance|row|
+-------+-----------+----+------+----------+---+
| Blessy|    Physics|  89|     P|        75|  1|
| Blessy|  Chemistry|  76|     P|        75|  2|
| Blessy|Mathematics|  63|     P|        75|  3|
|  David|Mathematics|  97|     P|        70|  1|
|  David|    Physics|  90|     P|        70|  2|
|  David|  Chemistry|  87|     P|        70|  3|
|   John|Mathematics|  52|     P|        60|  1|
|   John|    Physics|  33|     F|        60|  2|
|   John|  Chemistry|  28|     F|        60|  3|
|Michael|    Physics|  80|     P|        90|  1|
|Michael|Mathematics|  78|     P|        90|  2|
|Michael|  Chemistry|  67|     P|        90|  3|
|  Nancy|Mathematics|  75|     P|        80|  1|
|  Nancy|  Chemistry|  59|     P|        80|  2|
|  Nancy|    Physics|  30|     F|        80|  3|
+-------+-----------+----+------+----------+---+



- Get Top N rows per Group of Name

In [9]:
df3 = df2.filter(col("row") <= 1)
df3.show()

+-------+-----------+----+------+----------+---+
|   Name|    Subject|Mark|Status|Attendance|row|
+-------+-----------+----+------+----------+---+
| Blessy|    Physics|  89|     P|        75|  1|
|  David|Mathematics|  97|     P|        70|  1|
|   John|Mathematics|  52|     P|        60|  1|
|Michael|    Physics|  80|     P|        90|  1|
|  Nancy|Mathematics|  75|     P|        80|  1|
+-------+-----------+----+------+----------+---+



- Create rank with each group of Subject

In [12]:
windowDept = Window.partitionBy("Subject").orderBy(col("Mark").desc())

In [13]:
df4 = df.withColumn("row", row_number().over(windowDept)).orderBy("Name", "row")
df4.show()

+-------+-----------+----+------+----------+---+
|   Name|    Subject|Mark|Status|Attendance|row|
+-------+-----------+----+------+----------+---+
| Blessy|  Chemistry|  76|     P|        75|  2|
| Blessy|    Physics|  89|     P|        75|  2|
| Blessy|Mathematics|  63|     P|        75|  4|
|  David|  Chemistry|  87|     P|        70|  1|
|  David|Mathematics|  97|     P|        70|  1|
|  David|    Physics|  90|     P|        70|  1|
|   John|    Physics|  33|     F|        60|  4|
|   John|  Chemistry|  28|     F|        60|  5|
|   John|Mathematics|  52|     P|        60|  5|
|Michael|Mathematics|  78|     P|        90|  2|
|Michael|  Chemistry|  67|     P|        90|  3|
|Michael|    Physics|  80|     P|        90|  3|
|  Nancy|Mathematics|  75|     P|        80|  3|
|  Nancy|  Chemistry|  59|     P|        80|  4|
|  Nancy|    Physics|  30|     F|        80|  5|
+-------+-----------+----+------+----------+---+



- Get Top N rows per Group of SUbject

In [14]:
df5 = df4.filter(col("row") <= 1)
df5.show()

+-----+-----------+----+------+----------+---+
| Name|    Subject|Mark|Status|Attendance|row|
+-----+-----------+----+------+----------+---+
|David|  Chemistry|  87|     P|        70|  1|
|David|Mathematics|  97|     P|        70|  1|
|David|    Physics|  90|     P|        70|  1|
+-----+-----------+----+------+----------+---+



- Reverse Rank() to get Bottom N Rows per Group

In [18]:
windowDept = Window.partitionBy("Subject").orderBy(col("Mark"))

In [19]:
df6 = df.withColumn("row", row_number().over(windowDept)).orderBy("Name", "row")
df6.show()

+-------+-----------+----+------+----------+---+
|   Name|    Subject|Mark|Status|Attendance|row|
+-------+-----------+----+------+----------+---+
| Blessy|Mathematics|  63|     P|        75|  2|
| Blessy|  Chemistry|  76|     P|        75|  4|
| Blessy|    Physics|  89|     P|        75|  4|
|  David|  Chemistry|  87|     P|        70|  5|
|  David|Mathematics|  97|     P|        70|  5|
|  David|    Physics|  90|     P|        70|  5|
|   John|  Chemistry|  28|     F|        60|  1|
|   John|Mathematics|  52|     P|        60|  1|
|   John|    Physics|  33|     F|        60|  2|
|Michael|  Chemistry|  67|     P|        90|  3|
|Michael|    Physics|  80|     P|        90|  3|
|Michael|Mathematics|  78|     P|        90|  4|
|  Nancy|    Physics|  30|     F|        80|  1|
|  Nancy|  Chemistry|  59|     P|        80|  2|
|  Nancy|Mathematics|  75|     P|        80|  3|
+-------+-----------+----+------+----------+---+



- Get Bottom N Rows Per Group

In [20]:
df7 = df6.filter(col("row") <= 1)
df7.show()

+-----+-----------+----+------+----------+---+
| Name|    Subject|Mark|Status|Attendance|row|
+-----+-----------+----+------+----------+---+
| John|  Chemistry|  28|     F|        60|  1|
| John|Mathematics|  52|     P|        60|  1|
|Nancy|    Physics|  30|     F|        80|  1|
+-----+-----------+----+------+----------+---+

